In [1]:
import cv2
import pytesseract
import pyautogui
import numpy as np
import openai
from openai import OpenAI
import pyautogui

In [2]:
coordinate_list=[[880,667,1600,719], # 문제(전체화면, 광고X) 1번째 줄
                 [880,710,1600, 760], # 문제(전체화면, 광고X) 2번째 줄
                 [880,760,1600,808], # 문제(전체화면, 광고X) 3번째 줄
                 [975,1010,1600,1063], # 선지1
                 [975,1130,1574,1187], # 선지2
                 [975,1254,1574,1316], # 선지3
                 [975,1387,1574,1447], # 선지4
                 [892,837,1089, 886]] # 장르

def capture_screen():
    txt=[]
    screenshot = np.array(pyautogui.screenshot())
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

    for index, (x1, y1, x2, y2) in enumerate(coordinate_list):
        cropped_screen = screenshot[y1:y2, x1:x2]
        cropped_screen = cv2.cvtColor(cropped_screen, cv2.COLOR_BGR2GRAY)
        t, cropped_screen = cv2.threshold(cropped_screen, -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
        cropped_screen = cv2.bilateralFilter(cropped_screen, 5, 75, 75)
        cropped_screen = cv2.resize(cropped_screen, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)


        cropped_screen = cv2.erode(cropped_screen, k)
        if False:
            cv2.imshow("ex",cropped_screen)
            cv2.waitKey()
            cv2.destroyAllWindows()
        if t >= 80: # 빈 값이면 거르기 
            txt.append(pytesseract.image_to_string(cropped_screen, lang='jpn', config='--psm 6 --oem 1'))
    return txt
#capture_screen()

In [3]:
def extract(txt_list):
    # txt_list에 있는 각각의 요소에 "\n\x0c"를 떼서 문자열로 만들기
    cleaned_data = [element.replace('\n\x0c', '') for element in txt_list]
    Question, Nominee, Jenre="",[],""
    for i in range(len(cleaned_data)):
        if i < len(cleaned_data)-5:
            Question=Question+cleaned_data[i]
        elif i == len(cleaned_data)-1:
            Jenre = Jenre+cleaned_data[i]
        else:
            Nominee.append(cleaned_data[i])
    return Question, Jenre, Nominee

In [4]:
# 정답과 오답을 찾는 함수 만들기
# 캡쳐화면에 O.png가 있는 위치를 x,y로 반환하기
import time 

def find_answer_location():
    screenshot = np.array(pyautogui.screenshot())# 전체화면 캡처
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY) # O.png탐색
    result = cv2.matchTemplate(gray, cv2.imread('../Img/O.png', 0), cv2.TM_CCOEFF_NORMED)# 이미지 검출
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)# 최대값 찾기
    return max_loc     # 최대값 위치 반환

# 정답 찾기
def find_answer():
    while True :
        x,y = find_answer_location()
        if 940<=x<=1050 and 970<=y<=1094:
            return 1
        if 940<=x<=1050 and 1100<=y<=1210:
            return 2
        if 940<=x<=1050 and 1234<=y<=1341:
            return 3
        if 940<=x<=1050 and 1353<=y<=1484:
            return 4

#answer_num=find_answer()


In [5]:
#answer=Nominee[answer_num-1]
#answer

In [6]:
# txt_list = capture_screen()  : 화면 캡처 후 질문, 선지 데이터 받아옴
# Queston, Nominee = def extract(txt_list) : 데이터를 질문과 선지롤 추출 
# answer_num=find_answer() : 정답을 찾음
# answer=Nominee[answer_num-1] : 정답 선지 데이터 받아옴

In [7]:
# txt_list = capture_screen()
# Question, Nominee = extract(txt_list)
# answer_num=find_answer()
# answer=Nominee[answer_num-1]
# print(Question, Nominee, answer)

In [8]:
import pandas as pd
# 열 이름이 Q, K1, K2, K3, K4, A 인 데이터프레임 생성
df = pd.DataFrame(columns=['Q', 'Category','K1', 'K2', 'K3', 'K4', 'A'])

# # Questionm, Nominee, answer을 한 행으로 추가
# df.loc[len(df)] = [Question, Nominee[0], Nominee[1], Nominee[2], Nominee[3], answer]
# df

In [9]:
# df.loc[len(df)] = [Question, Nominee[0], Nominee[1], Nominee[2], Nominee[3], answer]

In [10]:
df.to_csv('data.csv', index=False)

In [11]:
# 학습 시작
import pandas as pd
df = pd.read_csv('data.csv')
def collect_data():
    while True:
        txt_list = capture_screen()
        if len(txt_list)<5:
            time.sleep(2)
            continue
        Question, Genre, Nominee = extract(txt_list)
        #pyautogui.click(1067, 1040) 
        answer_num=find_answer()
        
        answer=Nominee[answer_num-1]
        df.loc[len(df)] = [Question, Genre, Nominee[0], Nominee[1], Nominee[2], Nominee[3], answer]
        print(f"총 데이터 개수 : {len(df)}\n질문 : {Question}\n선지 : {Nominee}\n정답-장르 : {answer} -------- {Genre}")
        df.to_csv('data.csv', index=False)
        time.sleep(2)

In [13]:
collect_data()

총 데이터 개수 : 1
질문 : 次のうち、「食中毒」と同じ意味で使われる言葉は何2
선지 : ['食あたり', '食なし', '食あり', '食はずれ']
정답-장르 : 食あたり -------- 理系
총 데이터 개수 : 2
질문 : ものの順序を表す数のことを、特に何という ?         「
선지 : ['DEATH数', 'Q数', '序数', '破数']
정답-장르 : 序数 -------- 理系
총 데이터 개수 : 3
질문 : 日食のときに、太陽を隠す天体は円時
선지 : ['金星', '直星', '月', 'ダークマター']
정답-장르 : 月 -------- 理系
총 데이터 개수 : 4
질문 : 次のうち、インフルエンザの病原体はどれ2?
선지 : ['インフルエンザウイルス', 'インフルエンザ菌', 'インフルエンザアメーバ', 'インフルエンザ原申']
정답-장르 : インフルエンザウイルス -------- 理系
총 데이터 개수 : 5
질문 : ヒトの結腸の末端部分のことを、その形状から何という 2
선지 : ['R字結腸', 'Q字結腸', 'T字結腸', 'S字結腸']
정답-장르 : S字結腸 -------- 理系
총 데이터 개수 : 6
질문 : のア:さエミさりまっ0時 もうしよう)】たすずれ 2?
선지 : ['秒速5m', '秒速Scm', '秒速5Skm', '秒速5mm']
정답-장르 : 秒速5Skm -------- 理系
총 데이터 개수 : 7
질문 : アルファベットで表す時間の単位で、「秒」は「S」、分は「M」ですが、「時」は何 ?
선지 : ['F', 'L', 'N', 'H']
정답-장르 : H -------- 理系
총 데이터 개수 : 8
질문 : 「ビピタゴラスの定理」の別名を持つ「三在方の定理」が使われるのは、次のうちどんな図形 ?
선지 : ['台形', '直角三角形', '正方形', '円']
정답-장르 : 直角三角形 -------- 理系
총 데이터 개수 : 9
질문 : 「アカ」 !「アオ」 「ヒメ」などの種半がいる、オールのように変化した手足が特徴の上過申類は何 ?
선지 : ['ウミガメ', 'ヤマガメ', 'リクガメ', 'ヌメマガメ']
정답

KeyboardInterrupt: 